In [10]:
import pandas as pd


# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\Fastag Fraud Detection Datasets\FastagFraudDetection.csv")


print(df)


      Transaction_ID        Timestamp Vehicle_Type         FastagID  \
0                  1   1/6/2023 11:20         Bus   FTG-001-ABC-121   
1                  2   1/7/2023 14:55          Car  FTG-002-XYZ-451   
2                  3   1/8/2023 18:25   Motorcycle              NaN   
3                  4    1/9/2023 2:05        Truck  FTG-044-LMN-322   
4                  5   1/10/2023 6:35          Van  FTG-505-DEF-652   
...              ...              ...          ...              ...   
4995            4996   1/1/2023 22:18        Truck  FTG-445-EDC-765   
4996            4997  1/17/2023 13:43          Van  FTG-446-LMK-432   
4997            4998    2/5/2023 5:08        Sedan  FTG-447-PLN-109   
4998            4999  2/20/2023 20:34          SUV  FTG-458-VFR-876   
4999            5000   3/10/2023 0:59         Bus   FTG-459-WSX-543   

     TollBoothID Lane_Type Vehicle_Dimensions  Transaction_Amount  \
0          A-101   Express              Large                 350   
1        

In [13]:
import pandas as pd


# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\Fastag Fraud Detection Datasets\FastagFraudDetection.csv")


# REMOVE DUBLICATE ROWS
df.drop_duplicates(inplace=True)



# DROP ROWS WITH MISSING VALUES

df.dropna(inplace=True)

# CHECK MISSING VALUES
print("Missing Values:")
print(df.isnull().sum())

# PRINT UNIQUE VALUES FOR CATEGORICAL COLUMNS
print("\nUnique Values for Categorical Columns:")
for column in df.select_dtypes(include=['object']).columns:
    print(f"{column}: {df[column].unique()}")

# SHOW
print("\nDataFrame:")
print(df)

Missing Values:
Transaction_ID           0
Timestamp                0
Vehicle_Type             0
FastagID                 0
TollBoothID              0
Lane_Type                0
Vehicle_Dimensions       0
Transaction_Amount       0
Amount_paid              0
Geographical_Location    0
Vehicle_Speed            0
Vehicle_Plate_Number     0
Fraud_indicator          0
dtype: int64

Unique Values for Categorical Columns:
Timestamp: ['1/6/2023 11:20' '1/7/2023 14:55' '1/9/2023 2:05' ... '2/5/2023 5:08'
 '2/20/2023 20:34' '3/10/2023 0:59']
Vehicle_Type: ['Bus ' 'Car' 'Truck' 'Van' 'Sedan' 'SUV' 'Motorcycle']
FastagID: ['FTG-001-ABC-121' 'FTG-002-XYZ-451' 'FTG-044-LMN-322' ...
 'FTG-447-PLN-109' 'FTG-458-VFR-876' 'FTG-459-WSX-543']
TollBoothID: ['A-101' 'B-102' 'C-103' 'D-106']
Lane_Type: ['Express' 'Regular']
Vehicle_Dimensions: ['Large' 'Small' 'Medium']
Geographical_Location: ['13.059816123454882, 77.77068662374292'
 '13.042660878688794, 77.47580097259879'
 '12.84197701525119, 77.6754752817

In [16]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\Fastag Fraud Detection Datasets\FastagFraudDetection.csv")

# REMOVE DUPLICATE ROWS
df.drop_duplicates(inplace=True)

# DROP ROWS WITH MISSING VALUES
df.dropna(inplace=True)

# CONVERT CATEGORICAL TO NUMERICAL USING LABEL ENCODER
label_encoder = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = label_encoder.fit_transform(df[col])

# CONVERT STRING TO FLOAT
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

# HANDLING OUTLIERS USING Z SCORE
z_scores = zscore(df.select_dtypes(include=['int64', 'float64']))
abs_z_scores = abs(z_scores)
outlier_rows = (abs_z_scores > 3).all(axis=1)
df = df[~outlier_rows]

# NORMALIZE AND SCALE
scaler = MinMaxScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])

#TARGET VARIABLE
X = df.drop('Fraud_indicator', axis=1) 
y = df['Fraud_indicator']  

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PRINT THE PREPROCESSED DATAFRAME
print("Preprocessed DataFrame:")
print(df)


# CHECK SHAPE 
print("Training set - X shape:", X_train.shape)
print("Training set - y shape:", y_train.shape)
print("Testing set - X shape:", X_test.shape)
print("Testing set - y shape:", y_test.shape)




Preprocessed DataFrame:
      Transaction_ID  Timestamp  Vehicle_Type  FastagID  TollBoothID  \
0             0.0000   0.083105      0.000000  0.000449     0.000000   
1             0.0002   0.088595      0.166667  0.002022     0.333333   
3             0.0006   0.097579      0.833333  0.054157     0.666667   
4             0.0008   0.005990      1.000000  0.642697     0.333333   
5             0.0010   0.007237      0.666667  0.090337     0.000000   
...              ...        ...           ...       ...          ...   
4995          0.9992   0.002995      0.833333  0.583371     0.666667   
4996          0.9994   0.021213      1.000000  0.583820     0.333333   
4997          0.9996   0.380834      0.666667  0.585393     0.000000   
4998          0.9998   0.342151      0.500000  0.601348     0.333333   
4999          1.0000   0.402795      0.000000  0.602247     0.666667   

      Lane_Type  Vehicle_Dimensions  Transaction_Amount  Amount_paid  \
0           0.0                 0.0    

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from scipy.stats import zscore

# DATASET
df = pd.read_csv(r"C:\Users\nh013\Desktop\Fastag Fraud Detection Datasets\FastagFraudDetection.csv")

# REMOVE DUPLICATE ROWS
df.drop_duplicates(inplace=True)

# DROP ROWS WITH MISSING VALUES
df.dropna(inplace=True)

# CONVERT CATEGORICAL TO NUMERICAL USING LABEL ENCODER
label_encoder = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = label_encoder.fit_transform(df[col])

# CONVERT STRING TO FLOAT
for col in df.columns:
    if df[col].dtype == 'object':
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

# HANDLING OUTLIERS USING Z SCORE
z_scores = zscore(df.select_dtypes(include=['int64', 'float64']))
abs_z_scores = abs(z_scores)
outlier_rows = (abs_z_scores > 3).all(axis=1)
df = df[~outlier_rows]

# NORMALIZE AND SCALE
scaler = MinMaxScaler()
df[df.columns] = scaler.fit_transform(df[df.columns])

# TARGET VARIABLE
X = df.drop('Fraud_indicator', axis=1)
y = df['Fraud_indicator']

# SPLIT THE DATA INTO TRAIN AND TEST SET
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# RANDOMFOREST CLASSIFIRE
random_forest_model = RandomForestClassifier(random_state=42)

# TRAIN THE MODEL 
random_forest_model.fit(X_train, y_train)

#MAKE PREDICTION ON TEST SET
y_pred = random_forest_model.predict(X_test)

#EVALUATE THE MDOEL 
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# DETECTION RESULT
detection_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print("\nDetection Results:")
print(detection_results)


Accuracy: 0.9842873176206509

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.93      0.96       192
         1.0       0.98      1.00      0.99       699

    accuracy                           0.98       891
   macro avg       0.99      0.96      0.98       891
weighted avg       0.98      0.98      0.98       891


Detection Results:
      Actual  Predicted
939      1.0        1.0
4279     1.0        1.0
2634     1.0        1.0
4675     1.0        1.0
1405     0.0        0.0
...      ...        ...
280      0.0        0.0
2282     1.0        1.0
2905     1.0        1.0
3779     1.0        1.0
4767     0.0        0.0

[891 rows x 2 columns]
